In [1]:
# https://kanoki.org/2019/02/26/compare-two-excel-files-for-difference-using-python/

# Stock trade verification code

import numpy as np
import pandas as pd
import datetime

In [2]:
# 현재 날짜 가져오기
current_date = datetime.datetime.now()

# yyyymmdd 형식으로 변환
# td = current_date.strftime("%Y%m%d")
# td_tr1 = current_date.strftime("%m월").lstrip("0")
# td_tr2 = current_date.strftime("%d일").lstrip("0")
# td_tr = td_tr1 + td_tr2

td = "20230607"
td_tr = "6월7일"

print("Today : " + td)


Today : 20230608


In [3]:
oms_df = pd.read_excel("Z:/02.펀드/019. 일간매매내역/" + td + "_oms.xls", )
trader_df = pd.read_excel("Z:/02.펀드/019. 일간매매내역/" + td_tr + " 거래.xlsx", )

FileNotFoundError: [Errno 2] No such file or directory: 'Z:/02.펀드/019. 일간매매내역/20230608_oms.xls'

In [ ]:
# Short sell, Buy cover, Buy, Sell 매매구분을 위한 열 병합 및 값 변환
oms_df1 = oms_df[["펀드", "종목명", "매매유형", "매매구분", "체결수량", "체결단가", "체결금액"]]

oms_df1["매매구분"] = oms_df1[["매매유형", "매매구분"]].agg('_'.join, axis=1)

oms_df1["매매구분"] = oms_df1["매매구분"].replace(['일반_매수', '일반_매도', '차입_매수', '차입_매도'], ['Buy', 'Sell', 'Buy cover', 'Short sell'])

oms_df1 = oms_df1.drop(oms_df1.columns[[2]], axis=1)

# oms_df1 = oms_df1.replace(1, np.nan)
oms_df1 = oms_df1.dropna()

col_to_move = oms_df1.pop('매매구분')

oms_df1.insert(2, '매매구분', col_to_move)

C:\Users\jtlee\AppData\Local\Temp\ipykernel_19004\1968330389.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oms_df1["매매구분"] = oms_df1[["매매유형", "매매구분"]].agg('_'.join, axis=1)
C:\Users\jtlee\AppData\Local\Temp\ipykernel_19004\1968330389.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oms_df1["매매구분"] = oms_df1["매매구분"].replace(['일반_매수', '일반_매도', '차입_매수', '차입_매도'], ['Buy', 'Sell', 'Buy cover', 'Short sell'])


#둘 다 키값을 종목명(Key1) and 체결수량(Key2)으로 잡고 정렬한 다음 데이터프레임의 밸류들이 일치하는지 확인

In [ ]:
oms_df1

,펀드,종목명,매매구분,체결수량,체결단가,체결금액
0,하이일드,나노팀,Sell,360,27000,9720000
1,공모주1호,진영,Sell,0,0,0
2,공모주1호,진영,Sell,0,0,0
3,공모주1호,나라셀라,Sell,0,0,0
4,공모주2호,나노팀,Sell,1000,26825,26825000
5,공모주2호,나노팀,Sell,712,26984,19212800
6,코스닥벤처,POSCO홀딩스,Buy cover,268,382000,102376000
7,코스닥벤처,신풍제약,Short sell,3463,17196,59551220
8,코스닥벤처,제넥신,Short sell,6065,12328,74770800
9,코스닥벤처,진영,Sell,0,0,0


In [ ]:
oms_df2 = oms_df1.set_index(['펀드', '종목명', '매매구분'])

oms_df3 = oms_df2.sort_index(axis=0)

oms_df3 = oms_df3[(oms_df3 != 0).all(axis=1)]

oms_df4 = oms_df3.groupby(['펀드', '종목명', '매매구분']).agg({'체결수량': 'sum', '체결단가': 'mean', '체결금액': 'sum'})

In [ ]:
trader_df1 = trader_df[["펀드", "종목명", "매매구분", "체결수량", "체결단가", "체결금액"]]

In [ ]:
trader_df1 = trader_df1.dropna()

trader_df2 = trader_df1.set_index('펀드', '종목명', '매매구분')

trader_df3 = trader_df2.groupby(['펀드', '종목명', '매매구분']).agg({'체결수량': 'sum', '체결단가': 'mean', '체결금액': 'sum'})

C:\Users\jtlee\AppData\Local\Temp\ipykernel_19004\2518177081.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.set_index except for the argument 'keys' will be keyword-only.
  trader_df2 = trader_df1.set_index('펀드', '종목명', '매매구분')


In [ ]:
trader_df3 = trader_df3.sort_index(axis=0)

In [ ]:
trader_df3 = trader_df3.astype({'체결수량' : 'int', '체결단가' : 'int', '체결금액' : 'int'})
oms_df4 = oms_df4.astype({'체결수량' : 'int', '체결단가' : 'int', '체결금액' : 'int'})

In [ ]:
trader_df3['체결금액'] = trader_df3['체결금액'].abs()

trader_df3 = trader_df3.reset_index()
oms_df4 = oms_df4.reset_index()

try:
    trader_df3 = trader_df3[trader_df3['종목명'] != 'KOSPI200F']
except:
    pass
try:
    trader_df3 = trader_df3[trader_df3['종목명'] != 'KOSDAQ150F']
except:
    pass
try:
    trader_df3 = trader_df3[trader_df3['종목명'] != '코스피200F']
except:
    pass
try:
    trader_df3 = trader_df3[trader_df3['종목명'] != '코스닥150F']
except:
    pass
try:
    trader_df3 = trader_df3[trader_df3['종목명'] != '코스피F200']
except:
    pass
try:
    trader_df3 = trader_df3[trader_df3['종목명'] != '코스닥F150']
except:
    pass

In [ ]:
# trader_df3.compare(oms_df3)

In [ ]:
trader_df3 = trader_df3.reset_index()
oms_df4 = oms_df4.reset_index()

In [ ]:
trader_df3

,index,펀드,종목명,매매구분,체결수량,체결단가,체결금액
0,0,공모주2호,나노팀,Sell,1712,26904,46037608
1,1,멀티전략,POSCO홀딩스,Buy cover,80,382000,30560000
2,2,멀티전략,신풍제약,Short sell,1690,17199,29066310
3,3,멀티전략,제넥신,Short sell,4020,12328,49558560
4,4,코스닥벤처,POSCO홀딩스,Buy cover,268,382000,102376000
5,5,코스닥벤처,신풍제약,Short sell,3463,17196,59549748
6,6,코스닥벤처,유안타제10호스팩,Sell,1168,2155,2517040
7,7,코스닥벤처,제넥신,Short sell,6065,12328,74769320
8,8,하이일드,나노팀,Sell,360,27000,9720000


In [ ]:
oms_df4

,index,펀드,종목명,매매구분,체결수량,체결단가,체결금액
0,0,공모주2호,나노팀,Sell,1712,26904,46037800
1,1,멀티전략,POSCO홀딩스,Buy cover,80,382000,30560000
2,2,멀티전략,신풍제약,Short sell,1690,17199,29066190
3,3,멀티전략,제넥신,Short sell,4020,12328,49556780
4,4,코스닥벤처,POSCO홀딩스,Buy cover,268,382000,102376000
5,5,코스닥벤처,신풍제약,Short sell,3463,17196,59551220
6,6,코스닥벤처,유안타제10호스팩,Sell,1168,2155,2517100
7,7,코스닥벤처,제넥신,Short sell,6065,12328,74770800
8,8,하이일드,나노팀,Sell,360,27000,9720000


In [ ]:
result_df = oms_df4

In [ ]:
result_df[['종목명']] = oms_df4[['종목명']]
result_df[['매매구분']] = oms_df4[['매매구분']]
result_df[['체결수량']] = oms_df4[['체결수량']] - trader_df3[['체결수량']]
result_df[['체결단가']] = oms_df4[['체결단가']] - trader_df3[['체결단가']]
result_df[['체결금액']] = oms_df4[['체결금액']] - trader_df3[['체결금액']]

In [ ]:
result_df

,index,펀드,종목명,매매구분,체결수량,체결단가,체결금액
0,0,공모주2호,나노팀,Sell,0,0,192
1,1,멀티전략,POSCO홀딩스,Buy cover,0,0,0
2,2,멀티전략,신풍제약,Short sell,0,0,-120
3,3,멀티전략,제넥신,Short sell,0,0,-1780
4,4,코스닥벤처,POSCO홀딩스,Buy cover,0,0,0
5,5,코스닥벤처,신풍제약,Short sell,0,0,1472
6,6,코스닥벤처,유안타제10호스팩,Sell,0,0,60
7,7,코스닥벤처,제넥신,Short sell,0,0,1480
8,8,하이일드,나노팀,Sell,0,0,0


In [ ]:
# result_df.to_excel('Z:/02.펀드/019. 일간매매내역/recon_result/' + td + "_recon_result.xlsx", index=True)